In [1]:
import os
import json
from bs4 import BeautifulSoup

def convert_html_to_json(input_folder, output_folder):
    # 出力フォルダが存在しない場合は作成
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 入力フォルダ内のすべてのファイルを取得
    for filename in os.listdir(input_folder):
        if filename.endswith('.html'):
            input_path = os.path.join(input_folder, filename)

            try:
                # HTMLファイルを読み込み
                with open(input_path, 'r', encoding='utf-8') as file:
                    html_content = file.read()

                # BeautifulSoupを使用してHTMLコンテンツをパース
                soup = BeautifulSoup(html_content, 'html.parser')

                # 特定のタグを除外
                for script in soup(['script', 'style']):
                    script.decompose()

                # ページのテキストを抽出
                text_content = soup.get_text(separator='\n').strip()

                # クリーンなテキストをJSON形式で保存
                output_data = {
                    "content": text_content,
                    "metadata": {
                        "filename": filename
                    }
                }

                # 出力ファイルパスを設定
                json_filename = os.path.splitext(filename)[0] + '.json'
                output_path = os.path.join(output_folder, json_filename)

                # JSONファイルとして保存（既存ファイルを上書き）
                with open(output_path, 'w', encoding='utf-8') as json_file:
                    json.dump(output_data, json_file, ensure_ascii=False, indent=4)
                print(f"{filename} のクリーンなテキストコンテンツが {json_filename} として保存されました。")
            except Exception as e:
                print(f"ファイル {filename} の処理中にエラーが発生しました: {e}")

# 入力フォルダと出力フォルダのパス
input_folder_path = 'src/HTMLs'
output_folder_path = 'src/JSONs'

# HTMLファイルを処理してJSONファイルに変換
convert_html_to_json(input_folder_path, output_folder_path)


BTP All IN Program.html のクリーンなテキストコンテンツが BTP All IN Program.json として保存されました。
BTP, Analytics Cloud_ Service Map - Ad Hoc Advisory.html のクリーンなテキストコンテンツが BTP, Analytics Cloud_ Service Map - Ad Hoc Advisory.json として保存されました。
BTP, Analytics Cloud_ Service Map - Development Infrastructure & Administrative Readiness.html のクリーンなテキストコンテンツが BTP, Analytics Cloud_ Service Map - Development Infrastructure & Administrative Readiness.json として保存されました。
BTP, Analytics Cloud_ Service Map - New Feature Adoption Guidance.html のクリーンなテキストコンテンツが BTP, Analytics Cloud_ Service Map - New Feature Adoption Guidance.json として保存されました。
BTP, Analytics Cloud_ Service Map - System Administrator & End User Training.html のクリーンなテキストコンテンツが BTP, Analytics Cloud_ Service Map - System Administrator & End User Training.json として保存されました。
BTP, Analytics Cloud_ Service Maps - Success Check_ Business Goals & KPI Reporting.html のクリーンなテキストコンテンツが BTP, Analytics Cloud_ Service Maps - Success Check_ Business Goals & KPI Reporting.json として保

In [2]:
import os
import shutil

def organize_files(input_folder, output_folder):
    # フォルダパスを定義
    excel_folder = os.path.join(output_folder, 'Excels')
    pptx_folder = os.path.join(output_folder, 'PPTXs')
    pdf_folder = os.path.join(output_folder, 'PDFs')
    other_folder = os.path.join(output_folder, 'Others')

    # 必要なフォルダを作成
    os.makedirs(excel_folder, exist_ok=True)
    os.makedirs(pptx_folder, exist_ok=True)
    os.makedirs(pdf_folder, exist_ok=True)
    os.makedirs(other_folder, exist_ok=True)

    # 入力フォルダ内のすべてのファイルとサブフォルダを再帰的に処理
    for root, _, files in os.walk(input_folder):
        for filename in files:
            input_path = os.path.join(root, filename)

            # ファイル拡張子を取得
            file_extension = os.path.splitext(filename)[1].lower()

            if file_extension == '.xlsx':
                # ExcelファイルをExcelフォルダにコピー
                shutil.copy2(input_path, os.path.join(excel_folder, filename))
                print(f"Copied {filename} to Excel folder.")

            elif file_extension == '.pptx':
                # PPTXファイルをPPTXsフォルダにコピー
                shutil.copy2(input_path, os.path.join(pptx_folder, filename))
                print(f"Copied {filename} to PPTX folder.")

            elif file_extension == '.pdf':
                # PDFファイルをPDFフォルダにコピー
                shutil.copy2(input_path, os.path.join(pdf_folder, filename))
                print(f"Copied {filename} to PDF folder.")

            else:
                # その他のファイルをOtherフォルダにコピー
                shutil.copy2(input_path, os.path.join(other_folder, filename))
                print(f"Copied {filename} to Other folder.")

# 入力フォルダと出力フォルダのパスを指定
input_folder = './src/original_docs'
output_folder = './src'

# ファイルを整理
organize_files(input_folder, output_folder)


Copied Learning-Journey_PrS-BTP_01042023.pdf to PDF folder.
Copied SAP-Business-Technology-Platform-Coverage-Inside-SAP-PreferredII.pdf to PDF folder.
Copied BTP 2021 2022 Sprint.xlsx to Excel folder.
Copied PrefSuccess-BTP-ServiceAssetTypes.pdf to PDF folder.
Copied Jam Announcements Transfer _ Aug2022.docx to Other folder.
Copied Jam Announcements Transfer _ Aug2022.pdf to PDF folder.
Copied SAP Preferred Success for SAP Analytics Cloud Upcoming Events July 2022 Announcement.oft to Other folder.
Copied SAP Preferred Success for SAP Analytics Cloud Upcoming Events July 2022 Announcement.pdf to PDF folder.
Copied SAP Preferred Success for SAP Business Technology Platform Upcoming Events July 2022 Announcement.oft to Other folder.
Copied SAP Preferred Success for SAP Business Technology Platform Upcoming Events July 2022 Announcement.pdf to PDF folder.
Copied SAP Preferred Success for SAP Integrated Business Planning Upcoming Events July 2022 Announcement.oft to Other folder.
Copied SAP

In [3]:
import os
import json
import shutil
import pandas as pd
from PyPDF2 import PdfReader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()

# 各ファイルの変換関数
def excel_to_json(file_path):
    dfs = pd.read_excel(file_path, sheet_name=None)
    data = {}
    for sheet_name, df in dfs.items():
        data[sheet_name] = df.to_dict(orient='records')
    return data

def pdf_to_json(file_path):
    reader = PdfReader(file_path)
    data = {}
    for i, page in enumerate(reader.pages):
        data[f'page_{i+1}'] = page.extract_text()
    return data

def pptx_to_json(file_path):
    slides = UnstructuredPowerPointLoader(file_path).load()
    data = {"slides": [{"page_content": slide.page_content, "metadata": slide.metadata} for slide in slides]}
    return data

# ファイルをJSONに変換し、「all_JSONs」フォルダにコピー
def convert_files_to_json(input_folder, output_folder):
    converters = {
        "Excels": excel_to_json,
        "PPTXs": pptx_to_json,
        "PDFs": pdf_to_json,
    }

    all_json_folder = os.path.join(output_folder, 'all_JSONs')
    os.makedirs(all_json_folder, exist_ok=True)

    for folder_name, convert_func in converters.items():
        print(f"Processing folder: {folder_name}")  # フォルダ切り替え時の表示
        folder = os.path.join(input_folder, folder_name)
        if os.path.exists(folder):
            for file_name in os.listdir(folder):
                file_path = os.path.join(folder, file_name)
                json_file_path = os.path.join(all_json_folder, f"{os.path.splitext(file_name)[0]}.json")

                # 既にJSONファイルが存在する場合はスキップ
                if os.path.exists(json_file_path):
                    print(f"JSON file already exists, skipping: {json_file_path}")
                    continue

                json_data = convert_func(file_path)
                with open(json_file_path, 'w', encoding='utf-8') as json_file:
                    json.dump({"page_content": json_data, "metadata": {"file_path": file_path, "filename": file_name}}, json_file, ensure_ascii=False, indent=4)
                print(f"Converted and copied file: {file_path} to {json_file_path}")

    # 既存のJSONファイルをall_JSONsにコピー
    json_folder = os.path.join(input_folder, "JSONs")
    if os.path.exists(json_folder):
        for file_name in os.listdir(json_folder):
            file_path = os.path.join(json_folder, file_name)
            json_file_path = os.path.join(all_json_folder, file_name)

            # 既にJSONファイルが存在する場合はスキップ
            if os.path.exists(json_file_path):
                print(f"JSON file already exists, skipping: {json_file_path}")
                continue

            shutil.copy(file_path, all_json_folder)
            print(f"Copied existing JSON file: {file_path} to {all_json_folder}")

# 入力フォルダと出力フォルダのパスを指定
input_folder = './src'
output_folder = './src'

# ファイルをJSONに変換してコピー
convert_files_to_json(input_folder, output_folder)

Processing folder: Excels
JSON file already exists, skipping: ./src/all_JSONs/BTP 2021 2022 Sprint.json
Processing folder: PPTXs
JSON file already exists, skipping: ./src/all_JSONs/Capability Advisory Check for Data Intelligence Cloud Infosheet.json
JSON file already exists, skipping: ./src/all_JSONs/Capability Advisory Check Master Data Governance Infosheet.json
JSON file already exists, skipping: ./src/all_JSONs/Cloud Mindset Workshop_assessment delivery.json
JSON file already exists, skipping: ./src/all_JSONs/Cloud Mindset Workshop_full deck.json
JSON file already exists, skipping: ./src/all_JSONs/Design Discovery for SAP MDG CE Infosheet.json
JSON file already exists, skipping: ./src/all_JSONs/Design Discovery for SFSF.json
JSON file already exists, skipping: ./src/all_JSONs/Design Discovery.json
JSON file already exists, skipping: ./src/all_JSONs/DesignDiscoveryCheck-CALM_For_Alert-Job_-Automation-Monitoring.json
JSON file already exists, skipping: ./src/all_JSONs/DesignDiscoveryC